In [ ]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [ ]:
train=pd.read_csv('train.csv', encoding='ANSI')
test=pd.read_csv('test.csv', encoding="ANSI")
submission=pd.read_csv('sample_submission.csv', encoding="ANSI")

In [ ]:
#train.shape 122400 X 10
#60개의 건물 X 85일 24시간 =122400
train

# Train data EDA

#### 건물별 독립변수들의 시간에 따른 변화 살펴보기

In [ ]:
train['date_time_type']=list(map(pd.to_datetime,train['date_time']))

In [ ]:
train

##### 전력사용량 변화

In [ ]:
#한글 폰트 깨짐 방지
from matplotlib import font_manager, rc

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [ ]:
grouped = train[['date_time_type','num', '전력사용량(kWh)']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index('date_time_type', inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['전력사용량(kWh)'])
    
    plt.figure()
    df.plot()
    plt.title(index)



In [ ]:
##insight: 주말이 있는 회사가 있고 없는 회사가 있음. 카테고리로 분류하면 좋을 듯?

##### 기온변화

In [ ]:
grouped = train[['date_time_type','num', '기온(°C)']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index('date_time_type', inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['기온(°C)'])
    
    plt.figure()
    df.plot()
    plt.title(index)



##### 풍속변화

In [ ]:
grouped = train[['date_time_type','num', '풍속(m/s)']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index('date_time_type', inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['풍속(m/s)'])
    
    plt.figure()
    df.plot()
    plt.title(index)



##### 습도 변화

In [ ]:
grouped = train[['date_time_type','num', '습도(%)']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index('date_time_type', inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['습도(%)'])
    
    plt.figure()
    df.plot()
    plt.title(index)

##### 강수량 변화

In [ ]:
grouped = train[['date_time_type','num', '강수량(mm)']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index('date_time_type', inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['강수량(mm)'])
    
    plt.figure()
    df.plot()
    plt.title(index)



##### 일조량 변화

In [ ]:
grouped = train[['date_time_type','num', '일조(hr)']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index('date_time_type', inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['일조(hr)'])
    
    plt.figure()
    df.plot()
    plt.title(index)


#### 데이터 살펴보기

In [ ]:
train.head(30)

In [ ]:
train.info()

In [ ]:
test.info()

#### 건물별로 heatmap 

In [ ]:
train.columns

In [ ]:
grouped = train[['date_time_type','num', '전력사용량(kWh)', '기온(°C)', '풍속(m/s)', '습도(%)',
       '강수량(mm)', '일조(hr)']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index('date_time_type', inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['전력사용량(kWh)', '기온(°C)', '풍속(m/s)', '습도(%)',
       '강수량(mm)', '일조(hr)'])
    
    ax = plt.axes()
    sns.heatmap(df.corr(), annot=True, cmap='YlOrRd')
    ax.set_title(index)
    plt.show()
    





#### 건물별로 scatter plot

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
grouped = train[['date_time_type','num', '전력사용량(kWh)', '기온(°C)', '풍속(m/s)', '습도(%)',
       '강수량(mm)', '일조(hr)']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index('date_time_type', inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['num','전력사용량(kWh)', '기온(°C)', '풍속(m/s)', '습도(%)',
       '강수량(mm)', '일조(hr)'])
    sns.pairplot(df).set_title(index)
    plt.show()

    
    




In [ ]:
#test.shape 10080 X 9
#60개의 건물 X 7일 24시간 =10080
test

# **데이터 전처리**

In [ ]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']
    

In [ ]:
for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

In [ ]:
#날짜에서 월일시간추출
def time(x):
    return int(x[-2:])

train['month']=train['date_time_type'].apply(lambda x: x.month)
train['day']=train['date_time_type'].apply(lambda x: x.day)
train['time']=train['date_time'].apply(lambda x: time(x))

In [ ]:
train

In [ ]:
#건물별로 시간별 일조량 변화 살펴보기(groupby 월일로 해서 시간별 변화를 한 plot에 겹쳐서 그려보기)

In [ ]:
train.columns

In [ ]:
grouped = train[['month','day', 'time','일조(hr)','num']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index(['month','day'], inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['time','일조(hr)'])
    grouped2=df.groupby(df.index)
    
    fig = plt.figure()
    ax = fig.add_subplot() 
    for j in grouped2:
        
        j[1].set_index(['time'], inplace=True)
        df2=pd.DataFrame(j[1], index=j[1].index, columns=['일조(hr)'])
        
        ax.plot(df2.index, df2['일조(hr)'])
        #plt.xlim(0,18000)
        #plt.ylim(0,30)
    plt.title(index)    
    plt.show()

In [ ]:
#건물별로 시간별 전력량 변화 살펴보기(groupby 월일로 해서 시간별 변화를 한 plot에 겹쳐서 그려보기)

In [ ]:
train.columns

In [ ]:
grouped = train[['month','day', 'time','전력사용량(kWh)','num']].groupby('num')
for index, i in enumerate(grouped,1):
    i[1].set_index(['month','day'], inplace=True)
    df = pd.DataFrame(i[1], index= i[1].index, columns=['time','전력사용량(kWh)'])
    grouped2=df.groupby(df.index)
    
    fig = plt.figure()
    ax = fig.add_subplot() 
    for j in grouped2:
        
        j[1].set_index(['time'], inplace=True)
        df2=pd.DataFrame(j[1], index=j[1].index, columns=['전력사용량(kWh)'])
        
        ax.plot(df2.index, df2['전력사용량(kWh)'])
        
        #plt.xlim(0,18000)
        #plt.ylim(0,30)
    
    plt.title(index)    
    plt.show()

In [ ]:
## insight: 각 회사별로 기온에 따른 전력 사용량의 변화를 regression으로 구해서 그
##기울기를 독립변수로 추가해서 회사별 기온에 따른 예민도를 측정

In [ ]:
#시간과 전력사용량 간 상관관계 보기

In [ ]:
# grouped = train[['date_time_type','num', '전력사용량(kWh)', '기온(°C)', '풍속(m/s)', '습도(%)',
#        '강수량(mm)', '일조(hr)']].groupby('num')
# for index, i in enumerate(grouped,1):
#     i[1].set_index('date_time_type', inplace=True)
#     df = pd.DataFrame(i[1], index= i[1].index, columns=['전력사용량(kWh)', '기온(°C)', '풍속(m/s)', '습도(%)',
#        '강수량(mm)', '일조(hr)'])
    
#     ax = plt.axes()
#     sns.heatmap(df.corr(), annot=True, cmap='YlOrRd')
#     ax.set_title(index)
#     plt.show()

In [ ]:
#시간 변수와 요일 변수를 추가해봅니다.
def time(x):
    return int(x[-2:])
train['time']=train['date_time'].apply(lambda x: time(x))
test['time']=test['date_time'].apply(lambda x: time(x))

def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

#### 기온과 가장 상관관계가 높은 일조량으로 기온을 예측해서 test data의 결측치 보간

In [ ]:
test.info()

In [ ]:
model=LGBMRegressor(n_estimators=100)
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], 
         early_stopping_rounds=30, verbose=100)

+ test 결측치 보간해줍니다.
+ test 데이터의 변수는 예보 데이터이며, 예보 데이터는 train 데이터의 기간에 생성된 것이기에 활용 가능합니다.

In [ ]:
test = test.interpolate(method='values')

# **모델링**

In [ ]:
train_x=train.drop('전력사용량(kWh)', axis=1)
train_y=train[['전력사용량(kWh)']]

In [ ]:
train_x.drop('date_time', axis=1, inplace=True)
test.drop('date_time', axis=1, inplace=True)

In [ ]:
cross=KFold(n_splits=5, shuffle=True, random_state=1)
folds=[]
for train_idx, valid_idx in cross.split(train_x, train_y):
    folds.append((train_idx, valid_idx))

In [ ]:
folds

아래 코드는 최정명님의 코드 구성 방식을 살며시 활용했습니다. 감사합니다

https://www.dacon.io/competitions/official/235713/codeshare/2476?page=1&dtype=recent


In [ ]:
models={}
for fold in range(5):
    print(f'===================={fold+1}=======================')
    train_idx, valid_idx=folds[fold]
    X_train=train_x.iloc[train_idx, :]
    y_train=train_y.iloc[train_idx, :]
    X_valid=train_x.iloc[valid_idx, :]
    y_valid=train_y.iloc[valid_idx, :]
    
    model=LGBMRegressor(n_estimators=100)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], 
             early_stopping_rounds=30, verbose=100)
    models[fold]=model
    
    print(f'================================================\n\n')

In [ ]:
for i in range(5):
    submission['answer'] += models[i].predict(test)/5 

In [ ]:
submission

In [ ]:
#제출
submission.to_csv('baseline_submission3.csv', index=False)